In [12]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
import scipy.stats as st
from torch.optim import Adam
import argparse
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split


from collections import defaultdict
import pandas as pd
import numpy as np
import scanpy as scanpy

def _str2bool(v):
    return v.lower() in ("yes", "y", "true", "t", "1")


In [45]:

# dataloader.py에 있는 Custom_data()를 h5ad가 아니라 
# 이미 load된 AnnData 객체를 받아 처리하는 버전으로 하나 추가하면 됩니다.
def Custom_data_from_loaded(data, args):
    # 1. 라벨 매핑 정의
    id_dict = {
        'normal': 0,
        'COVID-19': 1,
        'hypertrophic cardiomyopathy':1,
        'dilated cardiomyopathy':2,

        'Healthy_stone_donor':0,
        'Healthy_living_donor':0,
        'CKD':1,
        'AKI':2
    }

    # 2. 환자 ID, 라벨, 셀 타입 정보 추출
    patient_id = data.obs['patient'] if 'patient' in data.obs else data.obs['donor_id']
    labels = data.obs['disease__ontology_label']  if 'disease__ontology_label' in data.obs else data.obs['disease_category']

    # cell_type = data.obs[args.cell_type_annotation]
    # 🔧 여기 수정: args.cell_type_annotation 우선 사용
    anno_col = getattr(args, "cell_type_annotation", "singler_annotation")
    if anno_col in data.obs:
        cell_type = data.obs[anno_col].astype("string").fillna("Unknown")
        print("cell type : ", cell_type)
    else:
        print(f"⚠️ '{anno_col}' 컬럼이 없어 'manual_annotation'로 대체합니다.")
        cell_type = data.obs['singler_annotation'].astype("string").fillna("Unknown")
    
    # cell_type = data.obs['manual_annotation']
    # cell_type = data.obs['singler_annotation']

    # pd.set_option('display.max_seq_items', None)  # 유니크 항목 출력 제한 해제
    # print("cell type annotation : ",cell_type)
    # print("✅ [DEBUG] manual_annotation 유니크값:", list(cell_type.unique()))
    # print("✅ [DEBUG] manual_annotation isna sum:", cell_type.isna().sum())
    # print("✅ [DEBUG] manual_annotation dtype:", cell_type.dtype)

    # print("✅ [DEBUG] NaN 위치들:")
    # print(cell_type[cell_type.isna()])

    # 3. expression 데이터 선택
    if args.pca:
        origin = data.obsm['X_pca']
    else:
        origin = data.X.toarray() if not isinstance(data.X, np.ndarray) else data.X

    # 4. 라벨을 숫자로 변환
    labels_ = np.array(labels.map(id_dict))

    # 5. 환자별 인덱스를 구성
    l_dict = {}
    indices = np.arange(origin.shape[0])
    p_ids = sorted(set(patient_id))
    p_idx = []
    
    
    # 환자 단위로 셀을 모아,
    # 다라벨이면 라벨별로 쪼개어 인덱스 묶음을 만들고,
    # 단일 라벨이면 환자 전체 셀 묶음을 만들어,
    # 이 묶음(=후속 단계에서 bag으로 쓰일 원천 집합) 들을 p_idx 리스트에 차곡차곡 쌓는 로직입니다.

    for i in p_ids: # 모든 환자 ID(p_ids)를 하나씩 순회합니다. 여기서 반복 변수 i는 “현재 환자 ID”입니다.
        idx = indices[patient_id == i] # 현재 환자 i에 속하는 셀들의 전체 인덱스를 뽑습니다.
                                        # patient_id == i가 불리언 마스크(길이 = 전체 셀 수)를 만들고,
                                        # 그 마스크로 indices를 필터링해 해당 환자의 셀 인덱스 배열 idx를 얻습니다.
        if len(set(labels_[idx])) > 1:   # one patient with more than one labels # 이 환자 i의 셀들(idx)에 서로 다른 라벨이 2개 이상 있는지 확인합니다.
            for ii in sorted(set(labels_[idx])): # 환자 i에서 라벨별로 나누어 처리하기 위해, 유일 라벨들을 정렬해서 하나씩 순회합니다. # 예: 라벨 집합이 {0, 1, -1}이라면 -1, 0, 1 순으로 돌아요.
                if ii > -1: # 유효 라벨만 사용합니다. # ex: 0,1,2
                    iidx = idx[labels_[idx] == ii] # 현재 라벨 ii에 해당하는 부분 셀 인덱스 묶음을 만듭니다.
                                                    # labels_[idx] == ii는 길이 len(idx)인 불리언 마스크,
                                                    # 그걸로 idx를 다시 필터링하면 “환자 i & 라벨 ii” 조건을 만족하는 셀 인덱스 배열 iidx가 됩니다.
                    tt_idx = iidx
                    # ★ 개수 체크(최소 셀 수 조건) 없이 전부 추가하기 위해 이 코드는 주석처리 하자
                    # if len(tt_idx) < 500:  # exclude the sample with the number of cells fewer than 500
                    #     continue
                    p_idx.append(tt_idx) # 라벨별로 쪼갠 인덱스 묶음(iidx) 을 p_idx 리스트에 넣습니다.
                                            # 나중 단계(샘플링/로더/모델)에서 이 묶음을 “한 bag의 원천 재료”로 사용합니다.
                                            # 다라벨 환자는 라벨 개수만큼 여러 묶음이 생깁니다.
                    l_dict[labels_[iidx[0]]] = l_dict.get(labels_[iidx[0]], 0) + 1     # p_idx에 하나의 인덱스 묶음(= 환자-라벨 그룹)을 추가할 때, 그 묶음의 라벨을 key로 해서 l_dict 값을 +1 증가

        else: # 이 분기는 단일 라벨 환자인 경우(= 유일 라벨 개수 == 1)입니다.
            if labels_[idx[0]] > -1: # 그 단일 라벨이 유효한지 확인합니다. (여기서도 -1은 제외)
                                    # 주의: idx가 비어있다면 idx[0]에서 에러가 납니다. 일반적으로 환자에 최소 1개 셀이 있다고 가정합니다.
                tt_idx = idx
                # ★ 개수 체크(최소 셀 수 조건) 없이 전부 추가하기 위해 이 코드는 주석처리 하자
                # if len(tt_idx) < 500:  # exclude the sample with the number of cells fewer than 500
                #     continue
                p_idx.append(tt_idx) # 단일 라벨 환자는 환자 전체 셀 인덱스 묶음(idx)을 그대로 p_idx에 추가합니다.
                                     # 이렇게 하면, 단일 라벨 환자는 묶음 1개, 다라벨 환자는 라벨 수만큼 여러 묶음이 들어가게 됩니다.
                l_dict[labels_[idx[0]]] = l_dict.get(labels_[idx[0]], 0) + 1     # p_idx에 하나의 인덱스 묶음(= 환자-라벨 그룹)을 추가할 때, 그 묶음의 라벨을 key로 해서 l_dict 값을 +1 증가

    print("라벨별 그룹 개수",l_dict) # 다라벨 환자는 라벨마다 여러 묶음으로 잡히니 그만큼 여러 번 카운트됩니다.
    # ex) 라벨별 그룹 개수 {1: 3, 2: 2, 0: 4} # https://chatgpt.com/s/t_689d6521e6088191bb414a646b947ee5 

    # 6. numpy 기반으로 반환
    return p_idx, labels_, np.array(cell_type), np.array(patient_id), origin


In [ ]:
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--seed', type=int, default=240)
parser.add_argument('--batch_size', type=int, default=16)
parser.add_argument('--learning_rate', type=float, default=3e-3)
parser.add_argument('--weight_decay', type=float, default=1e-4)
parser.add_argument('--epochs', type=int, default=100)
parser.add_argument("--task", type=str, default="severity")
parser.add_argument('--emb_dim', type=int, default=128)  # embedding dim
parser.add_argument('--h_dim', type=int, default=128)  # hidden dim of the model
parser.add_argument('--dropout', type=float, default=0.3)  # dropout
parser.add_argument('--layers', type=int, default=1)
parser.add_argument('--heads', type=int, default=8)
parser.add_argument("--train_sample_cells", type=int, default=500,
                    help="number of cells in one sample in train dataset") # 학습 시 각 환자 샘플에서 500개 세포를 랜덤 선택
parser.add_argument("--test_sample_cells", type=int, default=500,
                    help="number of cells in one sample in test dataset") # 테스트 시에도 동일하게 500개 세포 선택
parser.add_argument("--train_num_sample", type=int, default=20,
                    help="number of sampled data points in train dataset") # 한 명의 환자에서 500개의 세포를 20번 샘플링하여 20개의 bag 생성
parser.add_argument("--test_num_sample", type=int, default=100,
                    help="number of sampled data points in test dataset") # 테스트도 같은 방식으로 100개의 bag 생성
parser.add_argument('--model', type=str, default='Transformer')
parser.add_argument('--dataset', type=str, default=None)
parser.add_argument('--inter_only', type=_str2bool, default=False) # mixup된 샘플만 학습에 사용할지 여부
parser.add_argument('--same_pheno', type=int, default=0) # 같은 클래스끼리 mixup할지, 다른 클래스끼리 할지
parser.add_argument('--augment_num', type=int, default=0) # Mixup된 새로운 가짜 샘플을 몇 개 생성할지
parser.add_argument('--alpha', type=float, default=1.0) # mixup의 비율 (Beta 분포 파라미터)
parser.add_argument('--repeat', type=int, default=3)
parser.add_argument('--all', type=int, default=1)
parser.add_argument('--min_size', type=int, default=6000)
parser.add_argument('--n_splits', type=int, default=5)
parser.add_argument('--pca', type=_str2bool, default=False)
parser.add_argument('--mix_type', type=int, default=1)
parser.add_argument('--norm_first', type=_str2bool, default=False)
parser.add_argument('--warmup', type=_str2bool, default=False)
parser.add_argument('--top_k', type=int, default=1)

args, unknown = parser.parse_known_args()


In [46]:
from IPython.display import display
import pandas as pd
import numpy as np

# pandas 출력 무제한
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# numpy 출력 무제한
np.set_printoptions(threshold=np.inf)


In [47]:

for repeat in range(1):
    fold_aucs, accuracy, cms, recalls, precisions = [], [], [], [], []
    iter_count = 0
    for fold in range(1):
        print(f"🔁 Repeat {repeat}, Fold {fold}")
        train_path = f"/data/project/kim89/0804_data/repeat_{repeat}/fold_{fold}_train.h5ad"
        test_path = f"/data/project/kim89/0804_data/repeat_{repeat}/fold_{fold}_test.h5ad"

        train_data = scanpy.read_h5ad(train_path)
        test_data = scanpy.read_h5ad(test_path)

        train_p_index, train_labels, train_cell_type, patient_id, train_origin = Custom_data_from_loaded(train_data, args)
        test_p_index, test_labels, test_cell_type, test_patient_id, test_origin = Custom_data_from_loaded(test_data, args)

        # labels_ = train_labels
        labels_ = np.concatenate([train_labels, test_labels])

        print(f"🔍 Split #{iter_count + 1}")
        print(f"  → train_p_index 환자 수: {len(train_p_index)}")
        print(f"  → test_p_index 환자 수: {len(test_p_index)}")

        # 실제 환자 ID로 보기
        train_ids = [patient_id[idx[0]] for idx in train_p_index]
        test_ids = [test_patient_id[idx[0]] for idx in test_p_index]
        print(f"  → train 환자 ID: {train_ids}")
        print(f"  → test  환자 ID: {test_ids}")

        # 각 환자의 ID와 label 함께 출력
        print("  → train 환자 ID 및 라벨:")
        for idxs in train_p_index:
            idx = idxs[0]
            print(f"    ID: {patient_id[idx]}, Label: {train_labels[idx]}")

        print("  → test 환자 ID 및 라벨:")
        for idxs in test_p_index:
            idx = idxs[0]
            print(f"    ID: {test_patient_id[idx]}, Label: {test_labels[idx]}")


🔁 Repeat 0, Fold 0
cell type :  CATTAAGGATAT_COVID19_Participant13         Epithelial_cells
GTAGAAGGGGGG_COVID19_Participant13                Platelets
ATTAAGAGGTAG_COVID19_Participant13         Epithelial_cells
AGATATGACAAG_COVID19_Participant13         Epithelial_cells
CTTGTAACAAAT_COVID19_Participant13         Epithelial_cells
CACTATTTCTCT_COVID19_Participant13            Keratinocytes
CAAAATAGTCTA_COVID19_Participant13               Macrophage
GTTGTAATGGTT_COVID19_Participant13         Epithelial_cells
TACATAGAATAG_COVID19_Participant13                   B_cell
CATAGCTTATGT_COVID19_Participant13         Epithelial_cells
AACCAAGAAGTG_COVID19_Participant13              Neutrophils
GAACATAAGATT_COVID19_Participant13         Epithelial_cells
TCCGCACATAAA_COVID19_Participant13         Epithelial_cells
ATTATCGTCTTC_COVID19_Participant13                       DC
ATGAGAGCCAAA_COVID19_Participant13                 Monocyte
CATAGTTGTCGA_COVID19_Participant13         Epithelial_cells
GCACCGCA